In [1]:
from citylearn import  CityLearn, building_loader, auto_size
from energy_models import HeatPump, EnergyStorage, Building
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
np.random.seed(3)

import ray 
import ray.rllib.agents.ppo as ppo
from ray.tune.logger import pretty_print


import math
import random

import gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Normal


import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Use only one building for SINGLE AGENT environment, unmark multiple building IDs to simulate MULTI-AGENT environment. In the multi-agent environment
#the reward of each agent depend partially on the actions of the other agents or buildings (see reward_function.py)
building_ids = [8]#, 5, 9, 16, 21, 26, 33, 36, 49, 59]

In [3]:
'''
Building the RL environment with heating and cooling loads and weather files
CityLearn
    Weather file
    Buildings
        File with heating and cooling demands
        CoolingDevices (HeatPump)
        CoolingStorages (EnergyStorage)
'''

data_folder = Path("data/")

demand_file = data_folder / "AustinResidential_TH.csv"
weather_file = data_folder / 'Austin_Airp_TX-hour.csv'

heat_pump, heat_tank, cooling_tank = {}, {}, {}

#Ref: Assessment of energy efficiency in electric storage water heaters (2008 Energy and Buildings)
loss_factor = 0.19/24
buildings = []
for uid in building_ids:
    heat_pump[uid] = HeatPump(nominal_power = 9e12, eta_tech = 0.22, t_target_heating = 45, t_target_cooling = 10)
    heat_tank[uid] = EnergyStorage(capacity = 9e12, loss_coeff = loss_factor)
    cooling_tank[uid] = EnergyStorage(capacity = 9e12, loss_coeff = loss_factor)
    buildings.append(Building(uid, heating_storage = heat_tank[uid], cooling_storage = cooling_tank[uid], heating_device = heat_pump[uid], cooling_device = heat_pump[uid]))
    buildings[-1].state_space(np.array([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1, 40.0, 1.001]), np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0, 17.0, -0.001]))
    buildings[-1].action_space(np.array([0.2]), np.array([-0.2]))
    
building_loader(demand_file, weather_file, buildings)  
auto_size(buildings, t_target_heating = 45, t_target_cooling = 10)

env = CityLearn(demand_file, weather_file, buildings = buildings, time_resolution = 1, simulation_period = (3500,6000))

In [4]:
from reward_function import reward_function
observations_space, actions_space = [],[]
for building in buildings:
    observations_space.append(building.observation_spaces)
    actions_space.append(building.action_spaces)

In [5]:
def sigmoid(x):
    return(np.exp(x)/(1+np.exp(x)))


def derivative_log_sigmoid(x):
    der=(1/(sigmoid(x)-0.5))*sigmoid(x)*(1-sigmoid(x))
    return(x)

def montecarlo_return(rewards,gamma):
    l=len(rewards)
    r=0
    for i in reversed(range(l)):
        r=rewards[i]+(gamma**(l-i-1))*r
    return(r)



theta=np.random.rand(24)
w=np.random.rand(24)



In [ ]:
#reinforce montecarlo with baseline
#problem:The agent is not able to pick up the action
#action=tanh(xB)
from reward_function import reward_function
k = 0
cost, cum_reward = {}, {}
max_action=0.2
episodes = 10
alpha=0.1
gamma=0.9

for ep in range(10):
    rewards_=[]
    states=[]
    actions=[]
    
    state = env.reset()
    state=state[0][:24]
    states.append(state)
    done = False
    t=0
    #print(states)
    j=0
    while not done:
        #if k%500==0:
            #print('hour: '+str(k)+' of '+str(2500*episodes))
        x=np.matmul(theta.T,state.T)
        action = sigmoid(x)-0.5
        next_state, reward, done, _ = env.step([[action]])
        reward = reward_function(reward)[0] 
        
        actions.append(action)
        states.append(state)
        rewards_.append(reward)
        
        state = next_state[0][:24]
        
        t=t+1
        if done==True:
            T=t
        k+=1
        
    for t in range(T):
        G=montecarlo_return(rewards_[t:],1)
        s=states[t]
        x=np.matmul(theta.reshape(1,24),s.reshape(24,1))[0][0]
        v_s=np.matmul(w.reshape(1,24),s.reshape(24,1))[0][0]
        delta=G-v_s
        
        w=w+(alpha*delta*s)
        
        der=derivative_log_sigmoid(x)
        theta=theta+alpha*(gamma**t)*delta*der*s
    cost[ep] = env.cost()
    #print(theta)
    print(cost[ep])

In [7]:
#one step actor critic
def sigmoid(x):
    return(np.exp(x)/(1+np.exp(x)))


def derivative_log_sigmoid(x):
    der=(1/(sigmoid(x)-0.5))*sigmoid(x)*(1-sigmoid(x))
    return(x)

def montecarlo_return(rewards,gamma):
    l=len(rewards)
    r=0
    for i in reversed(range(l)):
        r=rewards[i]+(gamma**(l-i-1))*r
    return(r)


theta=np.random.rand(24)
w=np.random.rand(24)
     

In [16]:
from reward_function import reward_function
k = 0
cost, cum_reward = {}, {}
max_action=0.2
episodes = 10
alpha=0.1
gamma=0.9


for ep in range(10):
    
    rewards_=[]
    states=[]
    actions=[]
    
    state = env.reset()
    state=state[0][:24]
    #states.append(state)
    done = False
    I=1
    t=0
    while not done:
#         if k%500==0:
#             print('hour: '+str(k)+' of '+str(2500*episodes))
#         s=state
        x=np.matmul(theta.T,state.T)
        action = sigmoid(x)-0.5
        #print(action)
        next_state, reward, done, _ = env.step([[action]])
        reward = reward_function(reward)[0] 
        #print(reward)
        #actions.append(action)
        #states.append(state)
        #rewards_.append(reward)
        
        next_s=next_state[0][:24]
        v_nextstate=np.matmul(w.reshape(1,24),next_s.reshape(24,1))[0][0]
        v_state=np.matmul(w.reshape(1,24),s.reshape(24,1))[0][0]
        
        delta=reward+gamma*v_nextstate-v_state
        #print('del',delta)
        w=w+alpha*delta*s
        
        der=derivative_log_sigmoid(x)
        #print('der',der)
        theta=theta+alpha*I*delta*der*s
        I=gamma*I
        state = next_s
        
        t=t+1
        if done==True:
            T=t
        k+=1
   
    cost[ep] = env.cost()
    
    print(cost[ep])


186.79261669771415
186.79616611561832
186.79877198160565
186.80166149233835
186.804851109327
186.808358168026
186.81220079035586
186.8141681968384
186.807645416465
186.8240390508531


In [6]:
#one step actor critic
def sigmoid(x):
    return(np.exp(x)/(1+np.exp(x)))


def derivative_log_sigmoid(x):
    der=(1/(sigmoid(x)-0.5))*sigmoid(x)*(1-sigmoid(x))
    return(x)

def montecarlo_return(rewards,gamma):
    l=len(rewards)
    r=0
    for i in reversed(range(l)):
        r=rewards[i]+(gamma**(l-i-1))*r
    return(r)


theta=np.random.rand(24)
w=np.random.rand(24)
 

In [7]:
#actor critic with eligibility traces
from reward_function import reward_function
k = 0
cost, cum_reward = {}, {}
max_action=0.2
episodes = 10
alpha=0.5
gamma=0.9


for ep in range(1000):
    rewards_=[]
    states=[]
    actions=[]
    
    state = env.reset()
    state=state[0][:24]
    #states.append(state)
    done = False
    I=1
    t=0
    z_w=np.zeros(24)
    z_theta=np.zeros(24)
    R_bar=0
    lambda_w=0.8
    lambda_t=0.8
    while not done:
        #if k%500==0:
            #print('hour: '+str(k)+' of '+str(2500*episodes))
        s=state
        x=np.matmul(theta.T,state.T)
        action = sigmoid(x)-0.5
        #print(action)
        next_state, reward, done, _ = env.step([[action]])
        reward = reward_function(reward)[0] 
        #print(reward)
#         actions.append(action)
#         states.append(state)
#         rewards_.append(reward)
        
        next_s=next_state[0][:24]
        v_nextstate=np.matmul(w.reshape(1,24),next_s.reshape(24,1))[0][0]
        v_state=np.matmul(w.reshape(1,24),s.reshape(24,1))[0][0]
        
        delta=reward+gamma*v_nextstate-v_state-R_bar
        der=derivative_log_sigmoid(x)
        
        R_bar=R_bar+alpha*delta
        z_w=z_w*lambda_w*gamma+s
        z_theta=z_theta*lambda_t*gamma+der*s*I
        #print(delta)
        #print(alpha*delta*z_w)
        #print(alpha*I*delta*der*z_theta)
        w=w+alpha*delta*z_w
        theta=theta+alpha*I*delta*der*z_theta

        state = next_s
        
        t=t+1
        if done==True:
            T=t
        k+=1
   
    cost[ep] = env.cost()
    #print(theta)
    #if ep%100==0:
    print(cost[ep])



186.74175124980292
186.77020627103434
186.75040819468515
186.74051642953845
186.73443955076004
186.73067416594427
186.72919454980928
186.72972833531864
186.73189883896129
186.73502885309006
186.73889669302557
186.7426290927804
186.74587221113936
186.74929179582614
186.7526568957964
186.75654706522803
186.7599488840644
186.76316361590037
186.76617885501895
186.76830264948342
186.7712153063862
186.77463121943808
186.77838792776794
186.7804371838757
186.78028074699245
186.77966083534372
186.7791396369079
186.77898000106805
186.7791389065797
186.77806654834717
186.7771873512363


KeyboardInterrupt: 